<a href="https://colab.research.google.com/github/sindhuja279/BISlab/blob/main/Ant_Colony_BIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import random

class AntColony:
    def __init__(self, cities, num_ants, alpha, beta, rho, iterations):
        self.cities = cities
        self.num_cities = len(cities)
        self.num_ants = num_ants
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.iterations = iterations
        self.pheromone = np.ones((self.num_cities, self.num_cities))  # Initial pheromone levels
        self.distance_matrix = self.calculate_distance_matrix()
        self.best_route = None
        self.best_distance = float('inf')

    def calculate_distance_matrix(self):
        distance_matrix = np.zeros((self.num_cities, self.num_cities))
        for i in range(self.num_cities):
            for j in range(self.num_cities):
                distance_matrix[i][j] = np.linalg.norm(np.array(self.cities[i]) - np.array(self.cities[j]))
        return distance_matrix

    def choose_next_city(self, current_city, visited):
        probabilities = []
        for city in range(self.num_cities):
            if city not in visited:
                pheromone = self.pheromone[current_city][city] ** self.alpha
                heuristic = (1 / self.distance_matrix[current_city][city]) ** self.beta
                probabilities.append(pheromone * heuristic)
            else:
                probabilities.append(0)
        probabilities = np.array(probabilities)
        probabilities /= probabilities.sum()  # Normalize
        return np.random.choice(range(self.num_cities), p=probabilities)

    def construct_solution(self):
        for _ in range(self.num_ants):
            visited = [0]  # Start from the first city
            for _ in range(1, self.num_cities):
                next_city = self.choose_next_city(visited[-1], visited)
                visited.append(next_city)
            visited.append(0)  # Return to the starting city
            distance = self.calculate_route_distance(visited)
            if distance < self.best_distance:
                self.best_distance = distance
                self.best_route = visited

    def calculate_route_distance(self, route):
        return sum(self.distance_matrix[route[i]][route[i + 1]] for i in range(len(route) - 1))

    def update_pheromones(self):
        # Evaporate pheromones
        self.pheromone *= (1 - self.rho)
        # Add pheromones based on the best route found
        for city in range(len(self.best_route) - 1):
            self.pheromone[self.best_route[city]][self.best_route[city + 1]] += 1 / self.best_distance

    def optimize(self):
        for _ in range(self.iterations):
            self.construct_solution()
            self.update_pheromones()
        return self.best_route, self.best_distance

# Example usage:
if __name__ == "__main__":
    # Define cities as (x, y) coordinates
    cities = [(0, 0), (1, 2), (2, 4), (3, 1), (4, 3)]

    # ACO parameters
    num_ants = 10
    alpha = 1.0  # importance of pheromone
    beta = 2.0   # importance of heuristic
    rho = 0.5 # pheromone evaporation rate
    iterations = 10

    aco = AntColony(cities, num_ants, alpha, beta, rho, iterations)
    best_route, best_distance = aco.optimize()

    print("Best Route:", best_route)
    print("Best Distance:", best_distance)

Best Route: [0, 1, 2, 4, 3, 0]
Best Distance: 12.10654957016754
